# Neural Network using Pytorch library

In [18]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F 

## Check if hardware accelerators such as GPU or MPS are available, if not CPU

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('clean.csv')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['price']).values, df['price'].values, test_size=0.2, random_state=42)


In [35]:
from torch.utils.data import Dataset, DataLoader
import torch

class FlightDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32, device=device)
        self.y = torch.tensor(y, dtype=torch.float32, device=device).view(-1, 1)  # Ensure target is 2D for regression

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create PyTorch datasets and data loaders
train_dataset = FlightDataset(X_train, y_train)
test_dataset = FlightDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [36]:
import torch.nn.functional as F
class PricePredictor(nn.Module):
    def __init__(self, input_size):
        super(PricePredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Single output for regression

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [37]:
input_size = X_train.shape[1]  # Number of features

model = PricePredictor(input_size).to(device)
print(model)

criterion = nn.MSELoss()  # Regression loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

PricePredictor(
  (fc1): Linear(in_features=27, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)


In [38]:
epochs = 50

for epoch in range(epochs):
    model.train()  # Training mode
    total_loss = 0

    for inputs, targets in train_loader:
        # Forward pass
        predictions = model(inputs)
        loss = criterion(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/50], Loss: 654011607.8718
Epoch [2/50], Loss: 284760800.4061
Epoch [3/50], Loss: 185451464.6607
Epoch [4/50], Loss: 153308252.2297
Epoch [5/50], Loss: 122120603.2075
Epoch [6/50], Loss: 102661338.2333
Epoch [7/50], Loss: 95476334.0748
Epoch [8/50], Loss: 91838372.9528
Epoch [9/50], Loss: 89007751.5370
Epoch [10/50], Loss: 86502396.0784
Epoch [11/50], Loss: 84341111.2992
Epoch [12/50], Loss: 82308101.6848
Epoch [13/50], Loss: 80477427.4737
Epoch [14/50], Loss: 78688075.4221
Epoch [15/50], Loss: 77162853.9608
Epoch [16/50], Loss: 75724191.2556
Epoch [17/50], Loss: 74333673.1282
Epoch [18/50], Loss: 73029470.9154
Epoch [19/50], Loss: 71955487.8513
Epoch [20/50], Loss: 70916632.2191
Epoch [21/50], Loss: 70001483.0882
Epoch [22/50], Loss: 69216382.2271
Epoch [23/50], Loss: 68642574.2502
Epoch [24/50], Loss: 68055066.2582
Epoch [25/50], Loss: 67507354.4034
Epoch [26/50], Loss: 67097211.7640
Epoch [27/50], Loss: 66878261.6928
Epoch [28/50], Loss: 66559486.4087
Epoch [29/50], Loss: 66

In [39]:
model.eval()  # Evaluation mode
test_loss = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, targets in test_loader:
        predictions = model(inputs)
        loss = criterion(predictions, targets)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}")


Test Loss: 62038592.4662


In [42]:
new_data = torch.tensor(df.iloc[3], device=device)
predicted_price = model(new_data)
print(predicted_price.item)

C:\Users\User\AppData\Local\Temp\ipykernel_45016\3753619551.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_data = torch.tensor(df.iloc[3], device=device)


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float